In [1]:
import numpy as np
import pandas as pd
import folium 

from geopy import distance
from folium import FeatureGroup
from folium.plugins import HeatMap
from gestion_donnees import data_pollution

In [2]:
QA=pd.read_csv("indices_QA_commune_IDF_04_2018.csv")


In [3]:
##GRAPHE DE TEST POUR LA HEATMAP
# Nombre d'entrées
n = 200

# Coordonnées centrées sur Paris
latitude_center = 48.8566
longitude_center = 2.3522

# Générer des coordonnées proches de Paris
latitudes = np.round(np.random.normal(loc=latitude_center, scale=0.01, size=n), 6)
longitudes = np.round(np.random.normal(loc=longitude_center, scale=0.01, size=n), 6)

# Générer des valeurs aléatoires de PM2.5 entre 1 et 100
pm25_values = np.random.randint(1, 101, size=n)

# Créer la structure attendue
data = [[lat, lon, pm] for lat, lon, pm in zip(latitudes, longitudes, pm25_values)]

# Créer le DataFrame sous la forme demandée
test = pd.DataFrame(data, columns=['Latitude', 'Longitude', 'PM2.5'])

#test=pd.DataFrame([[48.86,2.356,10],[48.90,2.35,40],[48.87,2.357,60],[48.88,2.359,90]],columns=['Latitude','Longitude','PM2.5'])
test.head()

,Latitude,Longitude,PM2.5
0,48.871087,2.365003,9
1,48.866126,2.362420,75
2,48.845816,2.356577,77
3,48.858656,2.333548,56
4,48.859383,2.353714,13


In [4]:
L=[]
L.append(test.loc[0].values.tolist())
L.append(test.loc[1].values.tolist())
L


[[48.871087, 2.365003, 9.0], [48.866126, 2.36242, 75.0]]

In [5]:
import numpy as np
import pandas as pd

n = 300

# Coordonnées centrées sur Paris
latitude_center = 48.8566
longitude_center = 2.3522

# 1 degré de longitude ≈ 111 000 * cos(latitude) mètres
meters_per_degree_lon = 111_000 * np.cos(np.radians(latitude_center))

max_distance_m = 50

# Générer 299 distances aléatoires entre 0 et 50m (écarts entre points)
distances_m = np.random.uniform(0, max_distance_m, size=n-1)

# Calculer les décalages cumulés en mètres à partir du point de départ
cumulative_distances_m = np.insert(np.cumsum(distances_m), 0, 0)

# Convertir les distances cumulées en degrés de longitude
delta_lon = cumulative_distances_m / meters_per_degree_lon

# Les latitudes sont toutes égales à latitude_center
latitudes = np.full(n, latitude_center)

# Calculer les longitudes successives
longitudes = longitude_center + delta_lon

# Générer les valeurs PM
pm25_values = np.random.randint(1, 101, size=n)
pm1_values = np.random.randint(0, 101, size=n)
pm10_values = np.random.randint(0, 101, size=n)

# Ajouter time et speed = 1
times = np.ones(n, dtype=int)
speeds = np.ones(n, dtype=int)

# Créer la structure des données
data = [
    [time, lat, lon, speed, pm1, pm25, pm10]
    for time, lat, lon, speed, pm1, pm25, pm10 in zip(times, latitudes, longitudes, speeds, pm1_values, pm25_values, pm10_values)
]

# Créer le DataFrame
test = pd.DataFrame(data, columns=['Time','Latitude', 'Longitude', 'Speed','PM1', 'PM2.5', 'PM10'])

# Exporter en CSV
test.to_csv('points_alignes_paris.csv', index=False)

test.head()

,Time,Latitude,Longitude,Speed,PM1,PM2.5,PM10
0,1,48.8566,2.352200,1,15,33,100
1,1,48.8566,2.352555,1,9,22,97
2,1,48.8566,2.352927,1,14,25,91
3,1,48.8566,2.353236,1,76,42,72
4,1,48.8566,2.353528,1,26,22,31


In [6]:
data_pollution.head()

,Id,Time,Latitude,Longitude,Speed,PM1,PM2.5,PM10
0,5809,1970-01-01 00:00:00.001,48.8566,2.352200,1.0,45.0,57.0,58.0
1,5810,1970-01-01 00:00:00.001,48.8566,2.352645,1.0,12.0,53.0,89.0
2,5811,1970-01-01 00:00:00.001,48.8566,2.352823,1.0,19.0,33.0,12.0
3,5812,1970-01-01 00:00:00.001,48.8566,2.352935,1.0,68.0,36.0,50.0
4,5813,1970-01-01 00:00:00.001,48.8566,2.353211,1.0,34.0,16.0,85.0


In [7]:
#sélection selon la distance
data_drop=data_pollution.drop(columns=['Id','Time','Speed'])
#data_drop=test
n=len(data_drop)
Liste1=[]   #stocke les points intermédiaires
Mean=[]    #stocke les points moyens sur 50m
compteur=0   #la distance entre des points successifs
d_lim=50   #distance de 50m

for i in range(n-1):    
    coord1= (data_drop.loc[i,'Latitude'],data_drop.loc[i,'Longitude'])
    coord2=(data_drop.loc[i+1,'Latitude'],data_drop.loc[i+1,'Longitude'])
    d=distance.distance(coord1,coord2).m   # distance entre 2 coordonnées en mètres
    compteur += d    #ajoute la distance au nouveau point

    m=len(Liste1)
    if compteur <= d_lim :
        Liste1.append(data_drop.loc[i].values.tolist())
        
    else :
        if m==0:    #point isolé
            Mean.append(data_drop.loc[i].values.tolist())
            
        else :
            somme=[0,0,0,0,0]
            for ligne in Liste1:
                somme[0]+=ligne[0]
                somme[1]+=ligne[1]
                somme[2]+=ligne[2]
                somme[3]+=ligne[3]
                somme[4]+=ligne[4]
            Mean.append([val/m for val in somme])
        compteur=0
        Liste1=[]
        
data_50m = pd.DataFrame(Mean, columns=['Latitude', 'Longitude','PM1','PM2.5','PM10'])
len(data_50m)

110

In [8]:
len(data_pollution)
#len(data_50m)

300

In [9]:
data_1=data_50m.drop(columns=['PM2.5','PM10'])
data_2_5=data_50m.drop(columns=['PM1','PM10'])
data_10=data_50m.drop(columns=['PM1','PM2.5'])
len(data_1)

110

In [10]:
latitude = 48.8566
longitude = 2.3522
zoom=13
carte=folium.Map(location=[latitude,longitude],tiles="OpenStreetMap", zoom_start=zoom, min_zoom=12, max_zoom=18)

pollution = 42

#folium.Marker(location=[latitude,longitude], popup=f"Nox ={pollution} micro_g/m3", icon=folium.Icon(icon="cloud")).add_to(carte);

In [11]:
def niveau_couleur(value):
    if value <= 25 :
        return 'lightgreen'
    elif 25<value<=50 :
        return 'green'
    elif 50<value<=75 :
        return 'orange'
    else :
        return 'red'

In [12]:
#for index, row  in test.iterrows():
    #col=niveau_couleur(row['PM2.5'])
    #folium.Marker(location=[row['Latitude'],row['Longitude']], tooltip=f"Nox = {row['PM2.5']} micro_g/m3", icon=folium.Icon(icon="cloud",color=col)).add_to(carte);
    


In [13]:
heat1=HeatMap(data_1, gradient={0.0: 'lightgreen', 0.3: 'green', 0.6: 'orange', 1.0: 'red'}, min_opacity=0.1, name="PM1.0", show=True)
heat2_5=HeatMap(data_2_5, gradient={0.0: 'lightblue', 0.3: 'blue', 0.6: 'yellow', 1.0: 'red'}, min_opacity=0.1, name="PM2.5", show=False)
heat10=HeatMap(data_10, gradient={0.0: 'lightgreen', 0.3: 'green', 0.6: 'orange', 1.0: 'red'}, min_opacity=0.1, name="PM10", show=False)

In [14]:
carte.add_child(heat1)
carte.add_child(heat2_5)
carte.add_child(heat10)
folium.LayerControl(collapsed=False).add_to(carte);

In [15]:
carte.save("carte_pollution_3.html")

In [16]:
carte_test=folium.Map(location=[latitude,longitude],tiles="OpenStreetMap", zoom_start=zoom, min_zoom=12, max_zoom=15)

heat=HeatMap(test, name="PM2.5", gradient={0.0: 'lightgreen', 0.3: 'green', 0.6: 'orange', 1.0: 'red'}, show=True)
carte_test.add_child(heat);

heat2=HeatMap(test, name="PM1.0", gradient={0.0: 'lightblue', 0.3: 'blue', 0.6: 'lime', 1.0: 'red'}, show=False)
carte_test.add_child(heat2);

folium.LayerControl().add_to(carte_test)
display(carte_test)

In [17]:
display(carte)